Shows how one can generate text given a prompt and some hyperparameters, using either minGPT or huggingface/transformers

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [4]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2-xl'
device = 'auto'

In [5]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning

# ship model to device and set to eval mode
model.to(device)
model.eval();

number of parameters: 1557.61M


In [6]:

def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    # expand the first dimension to `num_samples`, and keep the rest the same (-1)
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    # the model will generate the next token at each forward pass, so we can get `steps` tokens in total
    # topk=40 restricts the sampling to the top 40 most likely next tokens, to keep the output more reasonable
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [7]:
generate(prompt='Andrej Karpathy, the', num_samples=10, steps=20)

downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /Users/yamei/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /Users/yamei/.cache/mingpt/vocab.bpe
--------------------------------------------------------------------------------
Andrej Karpathy, the director general of the ministry of health, said at a news conference on Thursday that the decision was only
--------------------------------------------------------------------------------
Andrej Karpathy, the leader of the Communist Party of Hungary and a close ally of the authoritarian right-wing government in Budapest
--------------------------------------------------------------------------------
Andrej Karpathy, the lead architect of the open-source C3 project, has posted a detailed description of how it works
--------------------------------------------------------------------------------
Andrej Karpathy, the founder and CEO of Bit